In [ ]:
seed_value= 123
import tensorflow as tf
import random as rn
import random

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
import tensorflow.keras
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten ,ZeroPadding2D
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv3D, MaxPooling3D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
from tensorflow.keras.initializers import Constant
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from scipy import ndimage

from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam,Adadelta, Adagrad
from tensorflow.keras import models
from tensorflow.keras import regularizers

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

from tensorflow.keras import activations
from tensorflow.keras import Model

from tensorflow.keras.layers import AvgPool2D, MaxPool2D, GlobalAvgPool2D
from tensorflow.keras.layers import Add, ReLU, Dense, concatenate

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, ZeroPadding2D,\
     Flatten, BatchNormalization, AveragePooling2D, Dense, Activation, Add

from tensorflow.keras.applications import VGG16,VGG19

In [ ]:
from keras.models import Model
from keras.layers import Dense
from keras.utils.np_utils import *
from keras.applications.resnet import ResNet50
from keras.applications.densenet import DenseNet121,DenseNet169,DenseNet201
from keras.applications.resnet import ResNet101, ResNet152
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.applications.resnet import preprocess_input
from tensorflow.keras.optimizers import SGD
import numpy as np
import os
from keras.callbacks import TensorBoard, ReduceLROnPlateau

In [ ]:
from sklearn.model_selection import LeaveOneOut, GridSearchCV
from sklearn.model_selection import KFold,StratifiedKFold

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

from statistics import mean, stdev
from sklearn.metrics import confusion_matrix, classification_report, cohen_kappa_score


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Dataset

In [ ]:

#only T2tra, ADC, Bval
label_data =pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Origin_Health/image_label.csv")

In [ ]:
label_data.head(5)

,Image_name,Plane
0,Patient01018_Plane3_2_of_3,Fetal brain
1,Patient00770_Plane3_2_of_3,Fetal brain
2,Patient00853_Plane3_1_of_3,Fetal brain
3,Patient01230_Plane3_5_of_5,Fetal brain
4,Patient01494_Plane3_3_of_4,Fetal brain


In [ ]:
img_dir = '/content/drive/MyDrive/Colab Notebooks/Origin_Health/images'
#test_dir = '/content/drive/MyDrive/Colab Notebooks/Origin_Health/External Test images'


In [ ]:
img_list = os.listdir(img_dir)
random.shuffle(img_list)
#test_img_list = os.listdir(test_dir)

In [ ]:
train_img_names = img_list[0:500] # taking randomly shuffled 500 images from 1646 images to reduce class imbalance
val_img_names = img_list[500:700]# 200 images for validation

**TEST** Iterating through the test image set to find out if every  test image has  a label image (goota plug this in somewhere in the end for actual testing)

In [ ]:
# test_final_names = []
# for i in label_data.Image_name:
#   for j in test_img_names:
#     if i == j.split('.')[0]:
#       #print(i)
#       #print(j.split('.')[0])
#       #print("we found a match, adding the image to our new test image set")
#       test_final_names.append(j)
# print(test_final_names)
# print(len(test_final_names))



Only 5 fetal Brain images in the test images set have labels, so we go ahead consider those images as our new test set

### Loading data as array for training and validation sets

In [ ]:
#Loading Training image data
def resize_image(image): #Resizing the images
  image = tf.image.resize(image, (661, 661))
  return image

train_imgs = []
val_imgs = []

for i in range(len(train_img_names)):
  train_img = load_img(os.path.join(img_dir, train_img_names[i]))
  train_img = img_to_array(train_img)
  train_img = resize_image(train_img)
  train_imgs.append(tf.convert_to_tensor(train_img))

In [ ]:
#Loading validation image data
for i in range(len(val_img_names)):
  val_img = load_img(os.path.join(img_dir, val_img_names[i]))
  val_img = img_to_array(val_img)
  val_img = resize_image(val_img)
  val_imgs.append(tf.convert_to_tensor(val_img))

#Labels:
Separate loop for adding train_labels snd validation_labels

In [ ]:
train_labels = []
val_labels= []
#To only add train_labels with train_img in label_data
for i in range(len(train_img_names)):
  for j in range(len(label_data.Image_name)):
    if train_img_names[i].split(".")[0] == label_data.Image_name[j]:
      train_class = label_data.Plane[i]
      train_labels.append(train_class)
for i in range(len(val_img_names)):
  for j in range(len(label_data.Image_name)):
    if val_img_names[i].split(".")[0] == label_data.Image_name[j]:
      val_class = label_data.Plane[i]
      val_labels.append(val_class)

In [ ]:
val_labels

['Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal brain',
 'Fetal 

Mapping is a dictionary with each organ string label encoded to integers

In [ ]:
organs = ['Fetal brain','Fetal femur','Fetal thorax','Fetal abdomen']
mapping = {}
for i in range(len(organs)):
  mapping[organs[i]] = i
print(mapping)


{'Fetal brain': 0, 'Fetal femur': 1, 'Fetal thorax': 2, 'Fetal abdomen': 3}


In [ ]:
train_labels[0]

'Fetal brain'

In [ ]:
for i in range(len(train_labels)):
  train_labels[i] = mapping[train_labels[i]]
print(train_labels)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
for i in range(len(val_labels)):
  val_labels[i] = mapping[val_labels[i]]


In [ ]:
print(val_labels)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
train_labels = to_categorical(train_labels, 6)
val_labels = to_categorical(val_labels, 6)

In [ ]:
print(train_labels)

[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]


In [ ]:
print(val_labels)

[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]


Now we have labels as One-Hot vectors

I'm going with an SGD optimizer

In [ ]:
epochs = 10
learning_rate = 0.01
decay_rate = 0
momentum = 0.9

from tensorflow.keras.optimizers.legacy import SGD

sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

In [ ]:
# Learning Rate Scheduler
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=0)


I have defined Train and Val data Generator sued to generate batches of images

Trying to input batch by batch using train_generator and validation_generator, now lets try datagen.from_tensor_slices

In [ ]:
# train_generator = train_datagen.flow(train_imgs, train_labels, batch_size=batch_size)
# image_numbers = len(train_imgs)

# validation_generator = test_datagen.flow(test_imgs, test_labels, batch_size=batch_size)

In [ ]:
train_loader = tf.data.Dataset.from_tensor_slices((train_imgs, train_labels))

In [ ]:
val_loader = tf.data.Dataset.from_tensor_slices((val_imgs, val_labels))


In [ ]:
batch_size = 8
# Augment the images on the fly during training.
train_dataset = (
    train_loader.shuffle(len(train_imgs))
    .batch(batch_size)
    .prefetch(2)
    )
# Only rescale.
val_dataset = (
    val_loader.shuffle(len(val_imgs))
    .batch(batch_size)
    .prefetch(2)
)

###
Visualize the Ultrasound image

In [ ]:
## Later please

In [ ]:
base_model = ResNet50(include_top=False, weights='imagenet', pooling='avg')

In [ ]:
predictions = Dense(4, activation='softmax')(base_model.output)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
image_numbers = len(train_imgs)
history = model.fit_generator(train_dataset, steps_per_epoch=10,
                                epochs=epochs,
                                validation_data=val_dataset,
                                validation_steps=batch_size,
                                )

Epoch 1/10


<ipython-input-55-748b4d261693>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_dataset, steps_per_epoch=10,


10/10 [==============================] - 5s 396ms/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 2/10
10/10 [==============================] - 3s 353ms/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 3/10
10/10 [==============================] - 3s 311ms/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 4/10
10/10 [==============================] - 3s 311ms/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 5/10
10/10 [==============================] - 3s 354ms/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 6/10
10/10 [==============================] - 3s 310ms/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 7/10
 3/10 [========>.....................] - ETA: 1s - loss: 0.0000e+00 - acc: 1.0000

10/10 [==============================] - 1s 130ms/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000


In [ ]:
save_dir = '/content/drive/MyDrive/Colab Notebooks/Origin_Health/results'

if not os.path.exists(save_dir):
  os.makedirs(save_dir)
model.save_weights(os.path.join(save_dir, 'Resnet50_weights_10_epoch.h5'))

history_dict = history.history

loss = history.history['loss']
np_loss = np.array(loss)
np.savetxt(os.path.join(save_dir,'train_loss_10_epoch.txt'), np_loss, fmt='%.3f')

val_loss = history.history['val_loss']
np_val_loss = np.array(val_loss)
np.savetxt(os.path.join(save_dir,'val_loss_10_epoch.txt'), np_val_loss, fmt='%.3f')

accy = history.history['acc']
np_accy = np.array(accy)
np.savetxt(os.path.join(save_dir, 'train_acc_10_epoch.txt'), np_accy, fmt='%.3f')

val_accy = history.history['val_acc']
np_val_accy = np.array(accy)
np.savetxt(os.path.join(save_dir, 'val_acc_10_epoch.txt'), np_val_accy, fmt='%.3f')
